### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x10d0fcb20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10ed0ffd0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage ## To provide message details to LLM human provides human message and system message is the instructions added to LLM how to act

messages = [
    SystemMessage(content = "Translate the following from English to French"),
    HumanMessage(content="Hello how are you?")
]

response = model.invoke(messages)

In [10]:
response.content

'Bonjour, je vais bien, merci. Et toi ?'

In [11]:
## To get output in a best format we use output parsers
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

'Bonjour, je vais bien, merci. Et toi ?'

In [12]:
## Using LCEL we can chain the components
chain = model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [16]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Transfer the following into {language}:"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user","{text}")
    ]
)

In [18]:
result = prompt.invoke({"language":"French",
                "text":"Hello"})

In [19]:
result.to_messages()

[SystemMessage(content='Transfer the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [20]:
chain = prompt|model|parser
chain.invoke({"language":"French",
                "text":"Hello"})

'Bonjour.'